In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer



In [7]:
import pandas as pd
import pickle 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np

with open("embeddings.pkl", 'rb') as f:
    embeddings = pickle.load(f)

embeddings = np.array(embeddings)

n_clusters =15
kmeans = KMeans(n_clusters=n_clusters , random_state=42)
clusters = kmeans.fit_predict(embeddings)

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

df = pd.DataFrame(reduced_embeddings, columns=['x', 'y'])
df['cluster'] = clusters

fig = px.scatter(df, x='x', y='y', color='cluster', title='Clusters')

fig.show()

In [9]:
import pandas as pd
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import numpy as np

# Load your scraped data
data = pd.read_json('articles.json')
data["text"] = data["title"] + " " + data["summary"]  # Combining title and summary for embedding

# Load embedding model (choose one: USE or Sentence Transformer)
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model_use = hub.load(module_url)
model_st = SentenceTransformer('all-mpnet-base-v2')  # Sentence Transformer alternative

def embed_text(text_data, model, model_type):
    if model_type == 'use':
        return model(text_data).numpy()
    elif model_type == 'sentence transformer':
        return model.encode(text_data)

# Generate embeddings using Universal Sentence Encoder
embeddings = embed_text(list(data['text']), model_use, 'use')

# Perform clustering
n_main_clusters = 5
kmeans_main = KMeans(n_clusters=n_main_clusters, random_state=42)
data['main_cluster'] = kmeans_main.fit_predict(embeddings)

# Sub-clustering within each main cluster
n_sub_clusters = 5  # Define sub-clusters per main cluster
sub_clusters = []

for main_cluster_id in range(n_main_clusters):
    # Select the articles in the main cluster
    main_cluster_data = embeddings[data['main_cluster'] == main_cluster_id]
    kmeans_sub = KMeans(n_clusters=n_sub_clusters, random_state=42)
    sub_cluster_labels = kmeans_sub.fit_predict(main_cluster_data)
    
    # Assign sub-cluster labels
    data.loc[data['main_cluster'] == main_cluster_id, 'sub_cluster'] = sub_cluster_labels

# Save data with new cluster columns to JSON
data.to_json('articles_with_clusters.json', orient='records')

# Prepare DataFrame for Plotly
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# Create DataFrame for Plotly
df_plot = pd.DataFrame(reduced_embeddings, columns=['x', 'y'])
df_plot['main_cluster'] = data['main_cluster'].astype(str)  # Convert to string for color mapping
df_plot['sub_cluster'] = data['sub_cluster'].astype(str)  # For marker mapping
df_plot['title'] = data['title']

# Plot using Plotly
fig = px.scatter(df_plot, x='x', y='y', color='main_cluster', symbol='sub_cluster', 
                 hover_data=['title'], title="Main and Sub-cluster Visualization")
fig.show()
